In [585]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

# Load data

In [586]:
path_to_data = '/Users/Enzo/Desktop/JMP/114402-V1/Replication-Fox-and-Bajari/data/'

### Bta network data

In [587]:
# ### airtravel data
# df_air_travel = pd.read_csv(path_to_data + 'air-travel-passengers-bta-year-1994.csv')
# print('airtravel        ', np.shape(df_air_travel))

# ### travel data
# df_travel_survery = pd.read_csv(path_to_data + 'american-travel-survey-1995-zero.csv')
# print('travel survey    ', np.shape(df_travel_survery))

# ### adjacency matrix
bta_adjacency = pd.read_csv(path_to_data + 'btamatrix_merged.csv',  header=None)
bta_adjacency = bta_adjacency.drop(bta_adjacency.columns[0], axis=1)
print('bta_adjacency ', np.shape(bta_adjacency))

# # compute fraction of 1s over number of entries
# bta_adjacency = bta_adjacency.drop(bta_adjacency.columns[0], axis=1)
# print('fraction of 1s: ', np.sum(bta_adjacency.values)/np.size(bta_adjacency.values))

### pop_wighted_centroid
pop_wighted_centroid = pd.read_csv(path_to_data + 'geographic-distance-population-weighted-centroid.csv')
# fill with zeros where its nan
pop_wighted_centroid = pop_wighted_centroid.fillna(0)
pop_wighted_centroid = np.vstack([pop_wighted_centroid, np.zeros(np.shape(pop_wighted_centroid)[1])])

print('pop_wighted_centroid     ', np.shape(pop_wighted_centroid))

# geographical distance
geo_distance = pd.read_csv(path_to_data + 'distancesmat_dio_perl_fixed.dat', delimiter=' ', header=None)
geo_distance = geo_distance.drop(geo_distance.columns[-1], axis=1)
print('geo_distance  ', np.shape(geo_distance))

bta_adjacency  (493, 493)
pop_wighted_centroid      (493, 493)
geo_distance   (493, 493)


### Bidder and bta data

In [588]:
### bta data
bta_data = pd.read_csv(path_to_data + 'btadata_2004_03_12_1.csv')
print('bta_data         ', np.shape(bta_data))

### bidder data
bidder_data = pd.read_csv(path_to_data + 'biddercblk_03_28_2004_pln.csv')

# drop FCC
bidder_data = bidder_data[bidder_data['co_name'] != 'FCC']

# give  0 eligibility to South Central Utah Telephone Association
bidder_data.loc[bidder_data['pops_eligible'].isnull(), 'pops_eligible'] = 0

print('bidder_data      ', np.shape(bidder_data))


def generate_matching_matrix():

    matching_matrix = np.zeros((len(bidder_data) , len(bta_data)), dtype= bool)

    for j in range(len(bta_data)):
        winner_id = bta_data['bidder_num_fox'].values[j] - 1
        matching_matrix[winner_id, j] = True
        
    return matching_matrix

matching_matrix = generate_matching_matrix()
print('Matching matrix: ', np.shape(matching_matrix))

bta_data          (493, 10)
bidder_data       (255, 4)
Matching matrix:  (255, 493)


### Other bta data 

In [667]:
bta_data

,bta,market,bid,winner,bidder_num_fox,pop90,percapin,hhinc35k,density,imwl
0,1,Aberdeen,411666.0,MCG,202,88891.0,14926.000,0.405000,7.536484,9
1,2,Aberdeen,471750.0,CookWest,184,83057.0,16374.000,0.433000,55.249180,3
2,3,"Abilene,",4024583.0,PokaLam,164,253174.0,15377.000,0.421000,54.129910,14
3,4,"Ada, OK",783000.0,OnQue,238,52677.0,12644.000,0.318000,32.020280,0
4,5,"Adrian,",701250.0,DCR,190,91476.0,16069.000,0.530000,256.705900,0
...,...,...,...,...,...,...,...,...,...,...
488,489,Mayaguez,29400000.0,PCS2000,182,1351600.0,3323.522,0.095447,1016.579000,0
489,490,Guam,1073250.0,DCR,190,133000.0,3345.715,0.095447,1504.793000,1
490,491,US Virgi,7797750.0,Windkeep,38,102000.0,3693.983,0.105383,1504.793000,1
491,492,American,411000.8,National,66,47000.0,3345.715,0.095447,1504.793000,0


In [589]:
bta_census = pd.read_csv(path_to_data + 'cntysv2000_census-bta-may2009.csv',encoding='ISO-8859-1')
print('bta_data         ', np.shape(bta_census))

bta_data          (494, 12)


### Form 175 data

In [590]:
bidder_data_form175 = pd.read_csv('/Users/Enzo/Desktop/JMP/form175_data/bidder_data_form175.csv')

relevant_columns = ['co_name', 'City', 'State','Applicant_Status', 
                    'Legal_Classification','bta']


# add to bidder_data the column bta from bidder_data_form175 by merging on 'co_name'
bidder_data = bidder_data.merge(bidder_data_form175[['co_name', 'bta', 'City', 'State','Applicant_Status', 
                                                    'Legal_Classification']], on='co_name', how='left')

# make bidder_data['bta'] integer
bidder_data['bta'] = bidder_data['bta'].astype(int)

# Estimation

In [591]:
num_agents = len(bidder_data) 
num_objects = len(bta_data)

print('Number of agents:  ', num_agents)
print('Number of objects: ', num_objects)

print('Matching matrix: ', np.shape(matching_matrix))

Number of agents:   255
Number of objects:  493
Matching matrix:  (255, 493)


In [ ]:
def round_weights(tick):
        
        capacity_i = bidder_data['pops_eligible'].to_numpy()
        weight_j = bta_data['pop90'].to_numpy()

        weight_j_rounded = (np.round((weight_j / tick) )).astype(int)
        capacity_i_rounded = (np.round((capacity_i / tick)) ).astype(int)

        return weight_j_rounded, capacity_i_rounded

weight_j, capacity_i = round_weights(50000)


print('max capacity: ', capacity_i.max())


### Characteristics

characteristics_list = []

# eligibility_i * pop_j  ù
US_pop_90 = weight_j.sum()
characteristic_i_j_1 = (capacity_i[:, None]/ US_pop_90 )* (weight_j[None, :]/ US_pop_90)
characteristic_i_j_1 = characteristic_i_j_1[:,:, None]

characteristic_i_j_1 /= characteristic_i_j_1.max()
characteristics_list.append(characteristic_i_j_1)

# geo_distance between bidder's bta and bta
bidder_bta = bidder_data['bta'].to_numpy() - 1
bta_id = bta_data['bta'].to_numpy() - 1

characteristic_i_j_2 = geo_distance.values[bidder_bta[:, None], bta_id[None, :]]
characteristic_i_j_2 = characteristic_i_j_2[:,:, None]

characteristic_i_j_2 /= characteristic_i_j_2.max()
characteristics_list.append(characteristic_i_j_2)

# Rural status_i * density_j
rural_i = bidder_data['Applicant_Status'].str.contains('Rural Telephone Company', na=False).to_numpy()
density_j = bta_data['density'].to_numpy()

characteristic_i_j_3 = rural_i[:, None] * density_j[None, :]
characteristic_i_j_3 = characteristic_i_j_3[:,:, None]

characteristic_i_j_3 /= characteristic_i_j_3.max()
characteristics_list.append(characteristic_i_j_3)

max capacity:  3612


In [666]:
%run ../Estimation_notebooks/linear_knapsack_DP.ipynb

# Create instance of the FCC auction
FCC_auction = BundledChoiceKP(num_agents, num_objects, num_simulations = 2, generate_data = False)

# Load data
characteristic_i_j_k = np.concatenate(characteristics_list, axis=2)
FCC_auction.load_data(characteristic_i_j_k, weight_j, capacity_i, matching_matrix)

# Run estimation
results, dual = FCC_auction.estimate_GMM_matching_batched( max_iters = 80 , tol = 1e-3)
# results, dual = FCC_auction.estimate_GMM_matching(tol = 1e-3)

mps:  True
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562382
Academic license 2562382 - for non-commercial use only - registered to ed___@nyu.edu
################
ITER: 0, parameters: [1000000. 1000000.]
Value of LP:    50438774.3769853
Reduced cost: 109916896.0
################
ITER: 1, parameters: [1000000.         -113167.39625297]
Value of LP:    1753673.2488577615
Reduced cost: 7936416.0
################
ITER: 2, parameters: [ 5.00775510e+00 -2.06547392e-03]
Value of LP:    -67.36340337363109
Reduced cost: 222.10365295410156
################
ITER: 3, parameters: [ 8.93989703 -0.91204767]
Value of LP:    -363.5652447582061
Reduced cost: 183.00970458984375
################
ITER: 4, parameters: [ 9.2392025  -1.45446027]
Value of LP:    -466.76604977156927
Reduced cost: 158.07081604003906
################
ITER: 5, parameters: [12.30378705 -2.04266961]
Value of LP:    -580.9544510210167
Reduced cost: 139.34786987304688
################
ITER: 6, 

In [657]:
results[1]

0.0

# Test supermodular knapsack Gurobi

In [567]:
# Create instance of the FCC auction
FCC_auction = BundledChoiceKP(num_agents, num_objects, [1], num_simulations = 2, generate_data = False)

# Load data
FCC_auction.load_data(characteristic_i_j_1, weight_j, capacity_i, matching_matrix)

#### Linear KP

In [568]:
id = 76

profit_j = characteristic_i_j_1[id,:,0] 
# profit_j = np.random.normal(0,10, num_objects)

W_total = capacity_i[id]
w_j = weight_j

with gp.Env(params = options) as env:
    with gp.Model(env=env) as model:
        # Add binary decision variables
        x = model.addVars(num_objects, vtype=GRB.BINARY, name="x")

        # Objective Function: Maximize total profit
        model.setObjective(gp.quicksum(profit_j[j] * x[j] for j in range(num_objects)), GRB.MAXIMIZE)

        # Constraint: Total weight must not exceed W_total
        model.addConstr(gp.quicksum(w_j[j] * x[j] for j in range(num_objects)) <= W_total, "WeightConstraint")

        # Optimize the model
        model.setParam('OutputFlag', 0)
        model.optimize()

        optimal_bundle = np.array(model.x)
        value = model.objVal


# Dynamic programming
B_i_star , value_dp = FCC_auction.linear_knapsack([id], FCC_auction.lambda_star, return_value = True)


# print the values 
print('Optimal value: ', value)
print('DP value: ', value_dp.item())

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562382
Academic license 2562382 - for non-commercial use only - registered to ed___@nyu.edu
Optimal value:  0.4864026664052533
DP value:  0.4864025115966797


#### Quadratic KP

In [715]:
num_objs = 400  # Define the size of the problem
capacity = 250 
lambda_k = np.array([1, 50])
i_id = 76
# Quadratic KP with random quadratic term

# profit_j = np.array(characteristic_i_j_1[id,:,0], dtype = float) 
np.random.seed(0)
profit_j = np.random.normal(0, 120, num_objs)

# profit_j = np.random.randint(0, 10, num_objs) 

# profit_j = (characteristic_i_j_k @ lambda_k)[i_id]
W_total = 300
w_j = np.random.randint(1, 100, num_objs)  
# w_j = weight_j
# print W_total and sum of w_j
print('W_total: ', W_total)
print('Sum of w_j: ', w_j.sum())


# Adjacency matrix
# density = 0.05
# upper_triangular = np.triu((np.random.rand(num_objs, num_objs) < density).astype(int), k=1)
# P_j_j = upper_triangular + upper_triangular.T
# np.fill_diagonal(P_j_j, 0)

# P_j_j *= 0
# P_j_j = np.random.rand(num_objs, num_objs)

# P_j_j = geo_distance.values
# P_j_j = pop_wighted_centroid / pop_wighted_centroid.max()
# P_j_j = np.random.rand(num_objs, num_objs) 
P_j_j = bta_adjacency.to_numpy()
# profit_j /= profit_j.mean()


with gp.Env(params=options) as env:
    with gp.Model(env=env) as model:
        # Add binary decision variables
        x = model.addVars(num_objs, vtype=GRB.BINARY, name="x")

        # Linear part of the objective
        linear_obj = gp.quicksum(profit_j[j] * x[j] for j in range(num_objs))

        # Quadratic part of the objective
        quadratic_obj = gp.quicksum(P_j_j[j, k] * x[j] * x[k] for j in range(num_objs) for k in range(num_objs))

        # Objective Function: Maximize total profit with quadratic term
        model.setObjective( lambda_k[0] * linear_obj + lambda_k[1] * quadratic_obj, GRB.MAXIMIZE)

        # Constraint: Total weight must not exceed W_total
        model.addConstr(gp.quicksum(w_j[j] * x[j] for j in range(num_objs)) <= W_total, "WeightConstraint")

        # Optimize the model
        # model.setParam('OutputFlag', 0)  
        model.optimize()

        # Extract results
        optimal_bundle = np.array([x[j].x for j in range(num_objs)])  # Decision variables
        value = model.objVal  # Objective value

W_total:  300
Sum of w_j:  19834
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562382
Academic license 2562382 - for non-commercial use only - registered to ed___@nyu.edu
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.1.0 24B83)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license 2562382 - for non-commercial use only - registered to ed___@nyu.edu
Optimize a model with 1 rows, 400 columns and 400 nonzeros
Model fingerprint: 0x2a8e41f1
Variable types: 0 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e-01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+02, 3e+02]
Found heuristic solution: objective 1963.9557494
Presolve removed 1 rows and 400 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work uni

In [714]:
optimal_bundle.sum()

27.0